RAINSTORM - Created on Oct 26, 2023 - @author: Santiago D'hers

### Geometric analysis
- This notebook will take the position.csv files (created using 1-Prepare_positions) and process them using geometric criteria (distance, angle and overall movement).

#### Requirements:
- The position.csv files.
- Geometric thresholds for exploration and freezing (see below).

---
#### 1. Load the necessary modules

In [7]:
import os
from glob import glob
import rainstorm.geometric_analysis as rst

---
#### 2. State your project path & thresholds
`base` : The path to the downloaded repository. If you are using a Windows path with backslashes, place an ‘r’ in front of the directory path to avoid an error (e.g. r'C:\Users\dhers\Desktop\RAINSTORM').

`filter_by` : A word that will be used to filter the position file used as an example.

`objects` : Name the stationary objects that may appear on your data.

`distance` : Maximum nose-object distance to consider exploration.

`angle` : Maximum head-object orientation angle to consider exploration.

`freezing_threshold` : Movement under which we consider freezing, calculated as the mean standard deviation for all bodyparts over one second.

`darting_threshold` : Movement over which we consider darting, calculated as the mean standard deviation for all bodyparts over one second.

`video_fps` : State the frames per second of the videos.

In [10]:
# State your path:
base = r'C:\Users\dhers\Desktop\RAINSTORM'
folder_path = os.path.join(base, r'docs\examples\NOR_example_copy')

all_position = glob(os.path.join(folder_path,"*/position/*position.csv"))
filter_by = 'TS'

# Set the parameters to evaluate behaviours
objects = ['obj_1', 'obj_2']
distance = 2.5 # in cm
angle = 45 # in degrees
freezing_threshold = 0.01
darting_threshold = 0.8

video_fps = 25

---
#### 3. We can open an example file and see what is inside

In [11]:
# Choose an example file to plot:
example_path = rst.choose_example(all_position, filter_by)

Plotting coordinates from NOR_TS_C2_A_position.csv


In [12]:
# Plot mouse exploration around multiple objects
rst.plot_position(example_path, objects = objects, maxDistance = distance, maxAngle = angle)

On the plot we have the position of each object in the example file. We can see the nose tracked and colored according to the angle of approach towards the objects.

We label exploration events when the nose is colored (heading towards the object) and inside the dashed line (close to the object).

---
#### 4.  We can also measure other behaviours, such as freezing

In [13]:
# Plot freezing events
rst.plot_freezing(example_path, fps = video_fps, threshold=freezing_threshold)

Now we are plotting the movement of the mouse. by setting a threshold (we can modify it above) we can quantify the time the mouse spent freezing.

---
#### 5. Finally, we can analyze the position files and create:
- A movement.csv file containing distance traveled, freezing and darting.
- A geolabels.csv file with the exploration of each object, frame by frame.

In [14]:
# Analyze all files in the folder
rst.create_movement_and_geolabels(all_position, objects, maxDistance = distance, maxAngle = angle, fps = video_fps, freezing_thr = freezing_threshold, darting_thr = darting_threshold)

NOR_Hab_C1_A_position.csv is missing objects: obj_1, obj_2
Saved movement to NOR_Hab_C1_A_movement.csv
NOR_Hab_C1_B_position.csv is missing objects: obj_1, obj_2
Saved movement to NOR_Hab_C1_B_movement.csv
NOR_Hab_C1_C_position.csv is missing objects: obj_1, obj_2
Saved movement to NOR_Hab_C1_C_movement.csv
NOR_Hab_C1_D_position.csv is missing objects: obj_1, obj_2
Saved movement to NOR_Hab_C1_D_movement.csv
NOR_Hab_C2_A_position.csv is missing objects: obj_1, obj_2
Saved movement to NOR_Hab_C2_A_movement.csv
NOR_Hab_C2_B_position.csv is missing objects: obj_1, obj_2
Saved movement to NOR_Hab_C2_B_movement.csv
NOR_Hab_C2_C_position.csv is missing objects: obj_1, obj_2
Saved movement to NOR_Hab_C2_C_movement.csv
NOR_Hab_C2_D_position.csv is missing objects: obj_1, obj_2
Saved movement to NOR_Hab_C2_D_movement.csv
NOR_Hab_C3_A_position.csv is missing objects: obj_1, obj_2
Saved movement to NOR_Hab_C3_A_movement.csv
NOR_Hab_C3_B_position.csv is missing objects: obj_1, obj_2
Saved movement